<a href="https://colab.research.google.com/github/Sagaust/DH-Computational-Methodologies/blob/main/Counting_Email_in_a_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import SQLite3 and Connect to Database:**
These lines import the sqlite3 module and create a connection to an SQLite database file named 'emaildb.sqlite'. If this file doesn't exist, it will be created. A cursor object (cur) is created to execute SQL commands.

In [17]:
import sqlite3
conn = sqlite3.connect('emaildb2.sqlite')
cur = conn.cursor()



## **Create or Reset Table:**
These commands drop (delete) the 'Counts' table if it already exists and then create a new 'Counts' table with two columns: 'email' (TEXT type) and 'count' (INTEGER type)

In [18]:
cur.execute('DROP TABLE IF EXISTS Counts')
cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')


## **File Input and Processing:**
The script prompts the user to enter a file name. If no name is entered, it defaults to 'mbox-short.txt'. It then opens this file for reading.

In [19]:
import os
print("Current working directory:", os.getcwd())


Current working directory: /content


In [20]:
fname = input('Enter file name: ')
if (len(fname) < 1): fname = '/content/drive/MyDrive/mbox.txt'
fh = open(fname)


Enter file name: 


In [23]:
for line in fh:
    if not line.startswith('From: '): continue
    pieces = line.split()
    email = pieces[1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (email,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (email,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (email,))
    conn.commit()

## **Parsing the File:**
The script iterates over each line of the file. It looks for lines that start with 'From: '. For each such line, it splits the line into parts and extracts the email address (which is assumed to be the second word in the line).

## **Database Interaction:**
For each extracted email, the script checks if that email is already in the 'Counts' table:

If not, it inserts the email with a count of 1.
If the email is already there, it updates the count by incrementing it.
This is done using SQL SELECT, INSERT, and UPDATE commands with parameters.

## **Committing Changes:**
After each insertion or update, the script commits the changes to the database using conn.commit(). This ensures that the changes are saved.

In [24]:
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

## **Query and Display Top 10 Emails:**
After processing the file, the script runs a SQL query to select the top 10 email addresses with the highest counts, ordered in descending order of count. It then prints out these email addresses and their counts.

In [25]:
for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

zqian@umich.edu 194
mmmay@indiana.edu 161
cwen@iupui.edu 158
chmaurer@iupui.edu 111
aaronz@vt.edu 110
ian@caret.cam.ac.uk 96
jimeng@umich.edu 93
rjlowe@iupui.edu 90
dlhaines@umich.edu 84
david.horwitz@uct.ac.za 67


 create a CSV file named email_counts.csv in the /content/ directory

In [26]:
import csv

In [27]:
# Fetch all data from the database
cur.execute('SELECT org, count FROM Counts')
rows = cur.fetchall()

In [28]:
# Write to a CSV file
with open('/content/drive/MyDrive/email_counts2.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['Org', 'Count'])  # Writing the headers
    csvwriter.writerows(rows)  # Writing data rows

In [30]:
# Close the cursor and connection
cur.close()
conn.close()

ProgrammingError: Cannot operate on a closed database.